In [1]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
load_dotenv()

AZURE_OPENAI_KEY = os.getenv("api_key")
AZURE_DEPLOYMENT_NAME = os.getenv("model-name")
API_VERSION = os.getenv("api-version")
AZURE_OPENAI_ENDPOINT = os.getenv("azure_endpoint")



# Create Azure Model Client
model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version=API_VERSION,
    deployment_name=AZURE_DEPLOYMENT_NAME ,  # Your deployment name
    model =AZURE_DEPLOYMENT_NAME 
)

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken


# Create an assistant agent
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a fun fact expert about animals.",
    model_client=model_client,
)

# Ask for a fun fact about dolphins
response = await assistant_agent.on_messages(
    [TextMessage(content="Tell me a fun fact about dolphins.", source="user")], CancellationToken()
)
print(response.chat_message.content)


Sure! Did you know that dolphins have names for each other? They use unique whistle patterns akin to individual names to identify and communicate with specific members of their pod. It's like their own equivalent of calling each other!


c:\Users\SanjeetKumar\AppData\Local\Programs\Python\Python313\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py:955: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


In [4]:

# Save the agent's state
agent_state = await assistant_agent.save_state()
print(agent_state)

{'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Tell me a fun fact about dolphins.', 'source': 'user', 'type': 'UserMessage'}, {'content': "Sure! Did you know that dolphins have names for each other? They use unique whistle patterns akin to individual names to identify and communicate with specific members of their pod. It's like their own equivalent of calling each other!", 'thought': None, 'source': 'assistant_agent', 'type': 'AssistantMessage'}]}}


In [5]:

# Create a new agent instance and load the saved state
new_assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a fun fact expert about animals.",
    model_client=model_client,
)
await new_assistant_agent.load_state(agent_state)

# Ask about the previous fun fact
response = await new_assistant_agent.on_messages(
    [TextMessage(content="What was the fun fact you shared about dolphins?", source="user")], CancellationToken()
)
print(response.chat_message.content)


The fun fact I shared is that dolphins have unique whistle patterns that act like names, allowing them to identify and communicate with specific members of their pod. It's like their own personalized calling system!


In [6]:
# Create an assistant agent
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a fun fact expert about animals.",
    model_client=model_client,
)

# Define a team that cycles through agents
agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))

# Ask for an animal fact
stream = agent_team.run_stream(task="Tell me a surprising fact about octopuses.")
await Console(stream)

# Save the team's state
team_state = await agent_team.save_state()

---------- TextMessage (user) ----------
Tell me a surprising fact about octopuses.


---------- TextMessage (assistant_agent) ----------
Certainly! Did you know that octopuses have **three hearts**? Two pump blood to the gills, while the third pumps it to the rest of the body. Here's the surprising part: the heart that pumps blood to the body actually stops beating when the octopus swims! This is one reason why octopuses are more prone to crawling than swimming—they're conserving energy when their main heart isn't working.


In [7]:
# Reset the team
await agent_team.reset()

# Try asking again
stream = agent_team.run_stream(task="What was the last animal fact you shared?")
await Console(stream)

---------- TextMessage (user) ----------
What was the last animal fact you shared?
---------- TextMessage (assistant_agent) ----------
Since I can't recall previous conversations, let me share a fresh animal fact with you! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. Interestingly, the heart that delivers blood to the body stops beating when they swim!


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last animal fact you shared?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=30, completion_tokens=65), metadata={}, content="Since I can't recall previous conversations, let me share a fresh animal fact with you! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. Interestingly, the heart that delivers blood to the body stops beating when they swim!", type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

In [7]:
# Load the saved state
await agent_team.load_state(team_state)

# Ask again
stream = agent_team.run_stream(task="What was the last animal fact you shared?")
await Console(stream)


---------- TextMessage (user) ----------
What was the last animal fact you shared?


---------- TextMessage (assistant_agent) ----------
The last animal fact I shared was about **octopuses** having three hearts, with one heart stopping when they swim to conserve energy.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last animal fact you shared?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=123, completion_tokens=29), metadata={}, content='The last animal fact I shared was about **octopuses** having three hearts, with one heart stopping when they swim to conserve energy.', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

In [8]:
import json

# Save state to a file
with open("team_state.json", "w") as f:
    json.dump(team_state, f)

# Load state from a file
with open("team_state.json", "r") as f:
    team_state = json.load(f)

# Create a new team and load the saved state
new_agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))
await new_agent_team.load_state(team_state)

# Ask again
stream = new_agent_team.run_stream(task="What was the last animal fact you shared?")
await Console(stream)


---------- TextMessage (user) ----------
What was the last animal fact you shared?
---------- TextMessage (assistant_agent) ----------
The last animal fact I shared was about octopuses having **three hearts** and how one of them stops beating when they swim! Let me know if you'd like another fun animal fact. 🐙


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last animal fact you shared?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=132, completion_tokens=42), metadata={}, content="The last animal fact I shared was about octopuses having **three hearts** and how one of them stops beating when they swim! Let me know if you'd like another fun animal fact. 🐙", type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')